**This code is inspired by https://huggingface.co/sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens**

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.8 MB/s 
     |████████████████████████████████| 61 kB 207 kB/s 
     |████████████████████████████████| 3.3 MB 45.0 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [4]:
#Mean Pooling - Take attention mask into account for correct averaging
# Perform pooling. In this case, max pooling.
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
   # print(token_embeddings.shape[-1])
   # print(tf.tile(tf.expand_dims(attention_mask,-1), multiples=(1,1,token_embeddings.shape[-1])))
    input_mask_expanded = tf.tile(tf.expand_dims(attention_mask,-1), multiples=(1,1,token_embeddings.shape[-1]))
    input_mask_expanded = tf.cast(input_mask_expanded, dtype=tf.float32)
    return tf.reduce_sum(token_embeddings * input_mask_expanded, 1) / tf.clip_by_value(tf.reduce_sum(input_mask_expanded, 1), clip_value_max=9999999999, clip_value_min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence',
             'how are you']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

# Compute token embeddings
with strategy.scope():
  model = TFAutoModel.from_pretrained("sentence-transformers/paraphrase-xlm-r-multilingual-v1", from_pt=True)
  model_output = model(encoded_input)

sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Sentence embeddings:
tf.Tensor(
[[ 0.11453557  0.07683478  0.02626467 ... -0.13231896 -0.00558202
   0.31623384]
 [ 0.13862178 -0.2031682   0.12559429 ...  0.05984864 -0.0185685
  -0.27948904]], shape=(2, 768), dtype=float32)
